### Selected Sentence Elimination From Youtube Deploy Result

In [22]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [23]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [24]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 1000 # 28  # native word end index

# word all usage in sent
word_use_big_than = 1

In [25]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_result(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_result(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [26]:
def simple_word_count_result(df, column):
    '''
    simple_word_count_result(df, column): df columns word count for word frequency\n
    df is dataframe, column is source for word count\n
    simple_word_count_result(df_file, "text"]):
    '''
    list_all = []    
    var_list = df[f"{column}"].dropna().tolist()
    for j in var_list:
        list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [27]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [28]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/4-Selected Sentence Elimination From Youtube Deploy Result"

Path(path).mkdir(parents=True, exist_ok=True)

In [29]:
# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment3/Result/Manuel"

# parameter
file = "Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6_Result_Manuel20"  # file without extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "Sent For 200 Words"

In [30]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987890,karneleme,5
987891,karnaya,5
987892,dörtlümüzün,5
987893,karnavalınız,5


In [31]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
995,995,resmi,68287
996,996,veriyor,68163
997,997,okul,68160
998,998,suçlu,68124


In [32]:
df_file = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name=f"{sheets}")
#df_file.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=True)
df_file.reset_index(drop=True, inplace=True)
df_file.sort_values("text",key=lambda x:x.str.len(), inplace=True, ascending=True)
#df_file = df_file.sample(235)
df_file

,video_id,start_time,end_time,text
315,37Nrnm53Cs4,714,716,kadar da sert ve sıkı
312,7CCGlFp8C98,407,410,ne cinayet ne de içki
314,lOa7Qp9JpWA,792,795,tam da bu sırada dava
313,sbzC2WeOkv0,755,757,o da tıpkı sizin gibi
311,Ri8OHAHmTJw,3413,3415,sizi baş göz mü edeyim
...,...,...,...,...
4,pGgYpcKg0Gs,363,369,ve hepsini beraber yaptığın arasında fark var ...
3,EDJh1X6oOXk,766,773,eğer olay şu ise son olarak onu söyleyeyim ya ...
2,sHnvEDmE3so,272,277,benim burada ne işim var neden bunu yapıyorum ...
1,cUAQrHbbpZ8,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...


In [33]:
word_count_result(df_file, df_file.columns[3:4])

,word,word_count
0,bir,110
1,bu,41
2,ne,40
3,çok,40
4,ben,39
...,...,...
876,haydi,1
877,hepsini,1
878,herhangi,1
879,herkesi,1


In [34]:
df_file_count = word_count_result(df_file, df_file.columns[3:4])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > word_use_big_than]
check_list = df_check["word"].to_list()
remove_list = []
df_file_var = df_file
for var_sent in df_file["text"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file_var = df_file_var[df_file_var["text"] != f"{var_sent}"]
        df_file_var_count = word_count_result(df_file_var, df_file_var.columns[3:4])
        df_check = df_file_var_count[df_file_var_count["word_count"] > word_use_big_than]
        check_list = df_check["word"].to_list()
        remove_list.append(var_sent)
    else:
        pass

df_file_var.sort_values("text",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_file_var.reset_index(drop=True, inplace=True)
df_file_var

,video_id,start_time,end_time,text
0,KsZ_8oIQsv4,319,325,başka biriyle herhangi bir şey yaşıyor musun k...
1,cUAQrHbbpZ8,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...
2,sHnvEDmE3so,272,277,benim burada ne işim var neden bunu yapıyorum ...
3,EDJh1X6oOXk,766,773,eğer olay şu ise son olarak onu söyleyeyim ya ...
4,pGgYpcKg0Gs,363,369,ve hepsini beraber yaptığın arasında fark var ...
...,...,...,...,...
268,Ri8OHAHmTJw,3413,3415,sizi baş göz mü edeyim
269,lOa7Qp9JpWA,792,795,tam da bu sırada dava
270,7CCGlFp8C98,407,410,ne cinayet ne de içki
271,sbzC2WeOkv0,755,757,o da tıpkı sizin gibi


In [35]:
df_remove_sent = pd.DataFrame(remove_list, columns=["remove_sent"])
df_remove_sent

,remove_sent
0,var ve oraya biz herşeyi
1,birçok şeyden de en ufak
2,o zaman bana öyle yapamam
3,ya şu taraftan çık ya çık
4,oldu hep o suçlu bu suçlu
5,merhaba orada kimse var mı
6,bir tür bir tür uyuşturucu
7,ama daha henüz hazır değil
8,tamam peki pekala pekala tamam
9,sonu belli olmayan tuhaf tuhaf


In [36]:
df_result_word_count = word_count_result(df_file_var, df_file.columns[3:4])
df_result_word_count

,word,word_count
0,bir,99
1,bu,38
2,ne,33
3,ben,33
4,çok,32
...,...,...
871,hayat,1
872,hayatta,1
873,hayatım,1
874,hayatını,1


In [37]:
df_result_word_count["word_count"].sum()

2207

In [38]:
sum_start_time = df_file_var["start_time"].sum()
sum_end_time = df_file_var["end_time"].sum()

In [39]:
(sum_end_time - sum_start_time)/60

15.75

In [40]:
df_file_var.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word_Result_Manuel_Master.xlsx", index=False)
df_remove_sent.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word_Result_Manuel_Remove.xlsx", index=False)

In [41]:
set_result_word_count = set(df_result_word_count["word"])
set_word_select = set(df_word_select["word"])

In [42]:
not_in_sent_word_list = list(set_word_select.difference(set_result_word_count))
not_in_sent_word_list

['berbat',
 'olmadığını',
 'yemeği',
 'sayın',
 'olmayacak',
 'pekala',
 'olası',
 'hangi',
 'neredeyse',
 'git',
 'adamı',
 'hepiniz',
 'sevindim',
 'gelecek',
 'suçlu',
 'canına',
 'öldürmek',
 'yaptın',
 'kapa',
 'söyledi',
 'nasılsın',
 'serbest',
 'karar',
 'iç',
 'gelen',
 'yanlış',
 'bul',
 'dün',
 'çekilin',
 'anlat',
 'sende',
 'tut',
 'adamlar',
 'gizli',
 'bulduk',
 'seninle',
 'birini',
 'pislik',
 'olayı',
 'bayım',
 'kimsenin',
 'dostum',
 'düşündüm',
 'durun',
 'adama',
 'geri',
 'sence',
 'halde',
 'miyim',
 'herif',
 'umurumda',
 'istemiyor',
 'çekil',
 'aferin',
 'tamamdır',
 'yaptığını',
 'burda',
 'kabul',
 'çek',
 'aldı',
 'doğum',
 'zavallı',
 'kayıp',
 'açık',
 'parti',
 'içine',
 'kes',
 'aldın',
 'tuhaf',
 'altında',
 'olma',
 'böylece',
 'gece',
 'hey',
 'duydum',
 'araba',
 'yemin',
 'babam',
 'beri',
 'sanıyorsun',
 'gitsin',
 'babası',
 'ol',
 'yolunda',
 'tehlikeli',
 'yakında',
 'acil',
 'nefes',
 'etme',
 'duydun',
 'dua',
 'çabuk',
 'haklı',
 'sebep',
 

#### Copy Move And Delete

In [43]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word_Result_Manuel*.xlsx") 
output_file

['Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word_Result_Manuel_Master.xlsx',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word_Result_Manuel_Remove.xlsx']

In [44]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [45]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

In [46]:
# Will improve because not reachable remove list
#def eliminate_sent(df_file, column, word_use_big_than):
#    '''
#    eliminate_sent(df_file, column, word_use_big_than)
#    '''    
#    df_file_count = simple_word_count_result(df_file, column)  
#    df_check = df_file_count[df_file_count["word_count"] > word_use_big_than]
#    check_list = df_check["word"].to_list()
#    remove_list = []
#    df_file_var = df_file
#    for var_sent in df_file[f"{column}"]:
#        result_list = []
#        words = re.findall(r"\w+",var_sent, re.UNICODE)
#        for word in words:
#            if word in check_list:
#                result_list.append(True)
#            else:
#                result_list.append(False)
#        if all(result_list):
#            df_file_var = df_file_var[df_file_var[f"{column}"] != f"{var_sent}"]
#            df_file_var_count = simple_word_count_result(df_file_var, column)
#            df_check = df_file_var_count[df_file_var_count["word_count"] > word_use_big_than]
#            check_list = df_check["word"].to_list()
#            remove_list.append(var_sent)
#        else:
#            pass
#
#    df_file_var.reset_index(drop=True, inplace=True)
#    
#    return df_file_var

In [47]:
#df_file_eliminate = eliminate_sent(df_file, "text", word_use_big_than)
#df_file_eliminate

In [48]:
#df_file_eliminate_word_count = simple_word_count_result(df_file_eliminate, "text")
#df_file_eliminate_word_count